# Implementación Path Relinking
---

In [3]:
from structure import instance, solution
from algorithms import grasp
from structure.solution import evaluate
from copy import deepcopy
import pandas as pd
import random

In [4]:
path = "instances/MDG-a_12_100_m10.txt"
inst = instance.readInstance(path)
sol, conjunto_soluciones, tiempo_final_grasp = grasp.execute(inst, 1, 0.1)

In [5]:
conjunto_soluciones_pd = pd.DataFrame(conjunto_soluciones)
conjunto_soluciones_pd = conjunto_soluciones_pd.drop_duplicates("sol")

In [6]:
conjunto_soluciones_pd.sort_values("of", ascending=False)

,instance,sol,of
5,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 96, 13, 15, 51, 20, 86, 55, 59, 93}",354.25
62,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{98, 2, 37, 90, 91, 12, 20, 26, 59, 63}",351.82
44,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 70, 43, 79, 16, 20, 86, 59, 60, 93}",350.18
46,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{5, 70, 39, 8, 74, 43, 14, 79, 17, 60}",349.53
68,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{66, 3, 14, 15, 78, 83, 53, 56, 61, 94}",349.45
...,...,...,...
530,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{97, 98, 3, 73, 44, 46, 82, 23, 57, 29}",320.14
527,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{96, 35, 5, 41, 42, 13, 22, 25, 26, 31}",319.35
317,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{67, 26, 72, 40, 75, 81, 85, 22, 24, 58}",317.13
191,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{96, 67, 4, 70, 73, 41, 83, 84, 22, 30}",312.50


In [7]:
sol_max = conjunto_soluciones_pd[conjunto_soluciones_pd["of"] == max(conjunto_soluciones_pd["of"])].iloc[0,1]
conjunto_soluciones_pd["n_distintos"] = conjunto_soluciones_pd["sol"].map(lambda x: len(x.difference(sol_max)))
conjunto_soluciones_pd[conjunto_soluciones_pd["n_distintos"] > 0].sort_values("of", ascending=False)

,instance,sol,of,n_distintos
62,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{98, 2, 37, 90, 91, 12, 20, 26, 59, 63}",351.82,8
44,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 70, 43, 79, 16, 20, 86, 59, 60, 93}",350.18,5
46,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{5, 70, 39, 8, 74, 43, 14, 79, 17, 60}",349.53,10
68,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{66, 3, 14, 15, 78, 83, 53, 56, 61, 94}",349.45,9
173,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{2, 98, 37, 15, 16, 20, 86, 59, 60, 93}",349.36,5
...,...,...,...,...
530,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{97, 98, 3, 73, 44, 46, 82, 23, 57, 29}",320.14,10
527,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{96, 35, 5, 41, 42, 13, 22, 25, 26, 31}",319.35,8
317,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{67, 26, 72, 40, 75, 81, 85, 22, 24, 58}",317.13,10
191,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{96, 67, 4, 70, 73, 41, 83, 84, 22, 30}",312.50,9


In [8]:
sol_0 = conjunto_soluciones[14]
sol_1 = conjunto_soluciones[57]

In [9]:
def transicion(sol_0, sol_1):
    best = None
    resultado = 0
    sol_2 = deepcopy(sol_0)
    sol_guia = deepcopy(sol_1)
    sol_guia["sol"] = sol_guia["sol"].difference(sol_0["sol"])
    for s0 in sol_0["sol"]:
        for s1 in sol_guia["sol"]:
            if s1 not in sol_0["sol"]:
                sol_2["sol"].remove(s0)
                sol_2["sol"].add(s1)
                sol_2["of"] = evaluate(sol_2)
                if best is None or sol_2["of"] > best["of"]:
                    anyadido = s1
                    best = deepcopy(sol_2)
                sol_2["sol"].remove(s1)
                sol_2["sol"].add(s0)
    
    sol_guia["sol"].remove(anyadido)

    if len(sol_guia["sol"]) > 1:
        resultado = transicion(best, sol_guia)

        if resultado["of"] < best["of"]:
            resultado = best
    else:
        print(best["sol"])
        resultado = best
    

    return resultado

best = transicion(sol_0, sol_1)

{67, 5, 70, 43, 14, 79, 17, 50, 60, 61}


In [10]:
best["of"]

339.2

In [11]:
sol_0["of"]

344.82

In [12]:
sol_1["sol"]

{13, 22, 24, 26, 58, 67, 72, 85, 90, 96}

### Opción para el Path Relinking

In [ ]:
def relink(inicio, guia):
    current = deepcopy(inicio)
    best = deepcopy(inicio)x

    to_add = guia["sol"] - current["sol"]
    to_remove = current["sol"] - guia["sol"]

    while to_add:
        vecinos = []
        for a in to_add:
            for r in to_remove:
                nuevo = deepcopy(current)
                nuevo["sol"].remove(r)
                nuevo["sol"].add(a)
                nuevo["of"] = evaluate(nuevo)
                vecinos.append(nuevo)
        if not vecinos:
            break
        current = max(vecinos, key=lambda s: s["of"])
        if current["of"] > best["of"]:
            best = deepcopy(current)

        to_add = guia["sol"] - current["sol"]
        to_remove = current["sol"] - guia["sol"]

    return best


In [14]:
# Seleccionamos las 5 mejores soluciones distintas (se puede modificar el nº de soluciones que se toman)
elite_df = conjunto_soluciones_pd.sort_values("of", ascending=False).drop_duplicates("sol").head(5)
elite_solutions = elite_df.to_dict("records")

# Aplicamos Path Relinking entre todos los pares de soluciones
mejor_pr = None
for i in range(len(elite_solutions)):
    for j in range(i+1, len(elite_solutions)):
        s1 = elite_solutions[i]
        s2 = elite_solutions[j]
        pr1 = relink(s1, s2)
        pr2 = relink(s2, s1)
        for cand in [pr1, pr2]:
            if mejor_pr is None or cand["of"] > mejor_pr["of"]:
                mejor_pr = cand

# Mostramos la mejor solución obtenida por Path Relinking
print("Mejor solución encontrada por Path Relinking:")
print("Conjunto:", mejor_pr["sol"])
print("Valor objetivo:", mejor_pr["of"])


Mejor solución encontrada por Path Relinking:
Conjunto: {64, 96, 13, 15, 51, 20, 86, 55, 59, 93}
Valor objetivo: 354.25000000000006
